# The chalet of the random gate
## Challenge statement
The Tensor Tundra's environment is so cold, it's an excellent place to build quantum devices. A safe haven for those who venture into these icy plains is the Chalet of the Random Gate.

The front entrance of the chalet is a quantum gate, and its action will take us to different entry points: if it's an X gate, it takes us through the backdoor, if it's a Y gate, it takes us through the chimney, and if it's a Z gate, we just go through the front door.

This gate is a programmable device, so that we can choose where to end up (for some reason, people tend to avoid the Y gate). But when a strong blizzard strikes, the controls become frozen solid. Instead of being stuck in one setting, the noise coming from the blizzard makes the gate act randomly.

In this challenge, we explore what happens when the application of a quantum gate is not done deterministically. Suppose that we have a single qubit |0⟩ on which a single gate acts at random. This gate can be:
* The Pauli X gate, acting with probability p,
* The Pauli Y gate, acting with probability q,
* The Pauli Z gate, acting with probability r,
* The Identity I, acting with probability 1−p−q−r.

You are asked to calculate the measurement probabilities in the computational basis upon the application of these random gates.

Fingers crossed that you don't get stuck in the chimney!
## Challenge Code
In the challenge template, you must complete the random_gate QNode which, given the probabilities p, q, and r (float) return the measurement probabilities (np.array(float)) in the computational basis.

## Input
As an input to this challenge, you are given the probabilities p, q and r (float) representing the application probabilities of the Pauli X, Y, and Z gates respectively.

## Output
The output is an np.tensor$[p_0,p_1]$ containing the probabilities of measuring 0 or 1 in the computational basis after the random application of the gate.

Test cases
The following public test cases are available for you to check your work. There are also some hidden test cases which we will use to check that your solution works in full generality.

test_input: [0.25,0.25,0.25]
expected_ouput: [0.5,0.5]

test_input: [0.125,0.25,0.2]
expected_ouput: [0.625, 0.375]


In [88]:
import json
import pennylane as qml
import pennylane.numpy as np

# Define your device
import random
dev = qml.device("default.mixed", wires=1)


@qml.qnode(dev)
def random_gate(p,q,r):

    """
    Applies a Pauli X, Pauli Y, Pauli Z or does nothing at random.

    Args:
        - p (float): probability of applying Pauli X.
        - q (float): probability of applying Pauli Y.
        - r (float): probability of applying Pauli Z.

    Returns:
        - (np.tensor(float)): Measurement probabilities in the computational basis.

    """

    # Calculate the Kraus operators for the bit flip channel
    K0 = np.sqrt(1-p-q-r) * np.eye(2)  # Identity matrix
    K1 = np.sqrt(p) * np.array([[0, 1], [1, 0]]) # X gate
    K2 = np.sqrt(q) * np.array([[0, -1j], [1j, 0]]) # Y gate
    K3 = np.sqrt(r) * np.array([[1, 0], [0, -1]]) # Z gate
    # Create custom channel
    qml.QubitChannel([K0,K1,K2,K3], wires=0)
    # Return the measurement probabilities in the computational basis
    return qml.probs(wires=0)
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    ins = np.array(json.loads(test_case_input))
    outs = random_gate(*ins).tolist()
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    assert np.allclose(solution_output, expected_output, rtol = 1e-4), "Not the correct probabilities"


# These are the public test cases
test_cases = [
    ('[0.25, 0.25, 0.25]', '[0.5, 0.5]'),
    ('[0.125, 0.25, 0.2]', '[0.625, 0.375]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.25, 0.25, 0.25]'...
Correct!
Running test case 1 with input '[0.125, 0.25, 0.2]'...
Correct!


Solution based on:
* [Quantum Error Correction](https://codebook.xanadu.ai/E.1)
* [Noisy circuits](https://pennylane.ai/qml/demos/tutorial_noisy_circuits/)
* [qml.operation.Channel](https://docs.pennylane.ai/en/stable/code/api/pennylane.operation.Channel.html)